# Importing Libraries

In [2]:
import pandas as pd
import numpy as np
from nltk.tokenize import RegexpTokenizer
from collections import Counter 
from sklearn import preprocessing
import re
import string
from nltk.corpus import stopwords
import codecs
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC
from sklearn.ensemble import VotingClassifier
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

# Importing Data

In [3]:
dataset = pd.read_csv("data.csv")
dataset.drop(['Unnamed: 0'], axis=1)

,article,label
0,قام مسئولو اتحاد الكرة بتغيير اللوجو الخاص بال...,sports
1,أعلن الاتحاد الأفريقي لكرة القدم، كاف، عن تفاص...,sports
2,أعلن نادى سيمبا التنزانى تعيين الفرنسى ديديه ج...,sports
3,حرص الجهاز الطبي للفريق الأول لكرة القدم بالنا...,sports
4,دائمًا ما تشهد غرفة خلع ملابس الأندية العديد م...,sports
...,...,...
5994,كشفت شركة أمازون خلال سبتمبر الماضى عن نظام دف...,tech
5995,على ما يبدو أن الهند ستكون الدولة التالية التي...,tech
5996,نسمع عن مصطلح الطيار الآلي، ولا نعلم كيف تعمل ...,tech
5997,يمكن مشاركة تغريدة من خلال تضمينها في موقع ويب...,tech


In [4]:
dataset['label'].unique()

array(['sports', 'economics', 'politics', 'celebrity', 'tech'],
      dtype=object)

# Tokenizing

In [4]:
tokenizer = RegexpTokenizer(r'\w+')
dataset["article"] = dataset["article"].apply(tokenizer.tokenize)

In [5]:
dataset = dataset.drop(['Unnamed: 0'], axis=1)

In [7]:
dataset.head()

,article,label
0,"[قام, مسئولو, اتحاد, الكرة, بتغيير, اللوجو, ال...",sports
1,"[أعلن, الاتحاد, الأفريقي, لكرة, القدم, كاف, عن...",sports
2,"[أعلن, نادى, سيمبا, التنزانى, تعيين, الفرنسى, ...",sports
3,"[حرص, الجهاز, الطبي, للفريق, الأول, لكرة, القد...",sports
4,"[دائم, ا, ما, تشهد, غرفة, خلع, ملابس, الأندية,...",sports


# Reading arabic stop words

In [6]:
def get_stop_words(path):
    stop_words = []
    with codecs.open(path, "r", encoding="utf-8", errors="ignore") as myfile:
        stop_words = myfile.readlines()
    stop_words = [word.strip() for word in stop_words]
    return stop_words
stop_words = get_stop_words('Arabic_stop_word.txt')

In [7]:
dataset["article"]=dataset["article"].apply(lambda x: [item for item in x if item not in stop_words])

In [10]:
dataset['article'].head()

0    [مسئولو, اتحاد, الكرة, بتغيير, اللوجو, الخاص, ...
1    [أعلن, الاتحاد, الأفريقي, لكرة, القدم, تفاصيل,...
2    [أعلن, نادى, سيمبا, التنزانى, تعيين, الفرنسى, ...
3    [حرص, الجهاز, الطبي, للفريق, الأول, لكرة, القد...
4    [دائم, تشهد, غرفة, خلع, ملابس, الأندية, العديد...
Name: article, dtype: object

# Printing the most common data in every label

In [11]:
all_words_sports = [word for tokens in dataset[dataset['label']=='sports']['article'] for word in tokens]
counter = Counter(all_words_sports)
counter.most_common(25)

[('الزمالك', 344),
 ('الفريق', 281),
 ('العالم', 234),
 ('لكرة', 231),
 ('مصر', 209),
 ('الأهلي', 187),
 ('اليد', 185),
 ('محمد', 177),
 ('فريق', 154),
 ('المدير', 152),
 ('الأول', 147),
 ('الكرة', 146),
 ('كأس', 146),
 ('مباراة', 139),
 ('الأهلى', 139),
 ('بطولة', 131),
 ('لاعب', 128),
 ('منتخب', 127),
 ('الفني', 115),
 ('مباريات', 107),
 ('القدم', 106),
 ('للأندية', 106),
 ('مهاجم', 104),
 ('نادى', 103),
 ('كشف', 102)]

In [12]:
all_words_economics = [word for tokens in dataset[dataset['label']=='economics']['article'] for word in tokens]
counter = Counter(all_words_economics)
counter.most_common(25)

[('2021', 289),
 ('1', 270),
 ('نقطة', 262),
 ('تعاملات', 253),
 ('العام', 250),
 ('مصر', 197),
 ('أسعار', 188),
 ('جلسة', 183),
 ('المصرية', 178),
 ('المصرى', 174),
 ('بنسبة', 163),
 ('مستوى', 154),
 ('0', 147),
 ('رئيس', 143),
 ('سعر', 140),
 ('الأسبوع', 136),
 ('جلسات', 136),
 ('الأربعاء', 128),
 ('الثلاثاء', 127),
 ('المؤشر', 118),
 ('وذلك', 118),
 ('التجارية', 107),
 ('بختام', 104),
 ('الخميس', 98),
 ('الذهب', 96)]

In [13]:
all_words_politics = [word for tokens in dataset[dataset['label']=='politics']['article'] for word in tokens]
counter = Counter(all_words_politics)
counter.most_common(25)

[('النواب', 689),
 ('مجلس', 364),
 ('لجنة', 322),
 ('رئيس', 305),
 ('بمجلس', 284),
 ('وزير', 249),
 ('الدكتور', 216),
 ('النائب', 213),
 ('برئاسة', 198),
 ('العامة', 194),
 ('قانون', 194),
 ('المستشار', 192),
 ('عبد', 142),
 ('محمد', 128),
 ('رقم', 120),
 ('لسنة', 119),
 ('الجلسة', 118),
 ('عضو', 117),
 ('الحكومة', 115),
 ('اللجنة', 113),
 ('جبالى', 105),
 ('حنفى', 104),
 ('مصر', 101),
 ('أكد', 99),
 ('المجلس', 94)]

In [14]:
all_words_celebrity = [word for tokens in dataset[dataset['label']=='celebrity']['article'] for word in tokens]
counter = Counter(all_words_celebrity)
counter.most_common(25)

[('الفنانة', 224),
 ('الفنان', 222),
 ('النجم', 222),
 ('النجمة', 214),
 ('محمد', 198),
 ('عبر', 155),
 ('مسلسل', 128),
 ('فيلم', 103),
 ('الجديد', 100),
 ('أحمد', 97),
 ('تصوير', 95),
 ('موقع', 95),
 ('عبد', 86),
 ('صورة', 78),
 ('الصغير', 78),
 ('العالمية', 74),
 ('كشف', 72),
 ('جديدة', 69),
 ('الراحل', 64),
 ('حسابها', 64),
 ('كورونا', 64),
 ('رمضان', 62),
 ('حسابه', 59),
 ('السابع', 59),
 ('انستجرام', 56)]

In [15]:
all_words_tech = [word for tokens in dataset[dataset['label']=='tech']['article'] for word in tokens]
counter = Counter(all_words_tech)
counter.most_common(25)

[('شركة', 510),
 ('الألعاب', 180),
 ('إيلون', 150),
 ('ماسك', 150),
 ('التنفيذي', 150),
 ('لشركة', 150),
 ('العام', 150),
 ('جديدة', 150),
 ('الخاص', 150),
 ('قالت', 150),
 ('الملياردير', 120),
 ('SpaceX', 120),
 ('جوجل', 120),
 ('استخدام', 120),
 ('تطبيق', 120),
 ('موقع', 120),
 ('أعلن', 120),
 ('إنها', 120),
 ('ارتداء', 120),
 ('الطيار', 120),
 ('التكنولوجي', 90),
 ('الجديدة', 90),
 ('Clubhouse', 90),
 ('كشفت', 90),
 ('أعلنت', 90)]

# Modeling

In [20]:
word_vectorizer_uni = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    stop_words=stop_words,
    ngram_range=(1, 1),
    max_features =1000)

In [21]:
unigramdataGet= word_vectorizer_uni.fit_transform(dataset['article'].astype('str'))
unigramdataGet = unigramdataGet.toarray()
vocab = word_vectorizer_uni.get_feature_names()
unigramdata=pd.DataFrame(np.round(unigramdataGet, 1), columns=vocab)
unigramdata[unigramdata>0] = 1

In [22]:
word_vectorizer_bi = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    stop_words=stop_words,
    ngram_range=(2, 2),
    max_features =1000)

In [23]:
BiagramdataGet= word_vectorizer_bi.fit_transform(dataset['article'].astype('str'))
BiagramdataGet= BiagramdataGet.toarray()

vocab1 = word_vectorizer_bi.get_feature_names()
Biagramdata=pd.DataFrame(np.round(BiagramdataGet, 1), columns=vocab1)
Biagramdata[Biagramdata>0] = 1

In [26]:
X=unigramdata
y=dataset['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=100)

In [27]:
svc=LinearSVC(C=1, max_iter=500)
svc= svc.fit(X_train , y_train)

y_pred1 = svc.predict(X_test)
dm=svc.score(X_test, y_test)
print('Accuracy score= {:.2f}'.format(svc.score(X_test, y_test)))

Accuracy score= 0.94


In [28]:
nab=GaussianNB(var_smoothing=1e-08)
nab= nab.fit(X_train , y_train)

y_pred1 = nab.predict(X_test)
nb=nab.score(X_test, y_test)
print('Accuracy score= {:.2f}'.format(nab.score(X_test, y_test)))

Accuracy score= 0.90


In [29]:
lor=LogisticRegression(C=2, max_iter=100)
lor= lor.fit(X_train , y_train)

y_pred1 = lor.predict(X_test)
lr=lor.score(X_test, y_test)
print('Accuracy score= {:.2f}'.format(lor.score(X_test, y_test)))

Accuracy score= 0.94


In [30]:
raf=RandomForestClassifier(min_samples_leaf=20, min_samples_split=20,random_state=100)
raf= raf.fit(X_train , y_train)

y_pred1 = raf.predict(X_test)
rf=raf.score(X_test, y_test)
print('Accuracy score= {:.2f}'.format(raf.score(X_test, y_test)))

Accuracy score= 0.87


In [31]:
Ens = VotingClassifier( estimators= [('SVM',svc),('NB',nab),('Log',lor),('raf',raf)], voting = 'hard')
Ens= Ens.fit(X_train , y_train)

y_pred1 = Ens.predict(X_test)
en=Ens.score(X_test, y_test)
print('Accuracy score= {:.2f}'.format(Ens.score(X_test, y_test)))

Accuracy score= 0.94


In [32]:
X=Biagramdata
y=dataset['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=100)

In [33]:
svc=LinearSVC(C=1, max_iter=500)
svc= svc.fit(X_train , y_train)

y_pred1 = svc.predict(X_test)
dm=svc.score(X_test, y_test)
print('Accuracy score= {:.2f}'.format(svc.score(X_test, y_test)))

Accuracy score= 0.72


In [34]:
nab=GaussianNB(var_smoothing=1e-08)
nab= nab.fit(X_train , y_train)

y_pred1 = nab.predict(X_test)
nb=nab.score(X_test, y_test)
print('Accuracy score= {:.2f}'.format(nab.score(X_test, y_test)))

Accuracy score= 0.71


In [38]:
lor=LogisticRegression(C=2, max_iter=100)
lor= lor.fit(X_train , y_train)

y_pred1 = lor.predict(X_test)
lr=lor.score(X_test, y_test)
print('Accuracy score= {:.2f}'.format(lor.score(X_test, y_test)))

Accuracy score= 0.72


In [36]:
raf=RandomForestClassifier(min_samples_leaf=20, min_samples_split=20,random_state=100)
raf= raf.fit(X_train , y_train)

y_pred1 = raf.predict(X_test)
rf=raf.score(X_test, y_test)
print('Accuracy score= {:.2f}'.format(raf.score(X_test, y_test)))

Accuracy score= 0.56


In [37]:
Ens = VotingClassifier( estimators= [('SVM',svc),('NB',nab),('Log',lor),('raf',raf)], voting = 'hard')
Ens= Ens.fit(X_train , y_train)

y_pred1 = Ens.predict(X_test)
en=Ens.score(X_test, y_test)
print('Accuracy score= {:.2f}'.format(Ens.score(X_test, y_test)))

Accuracy score= 0.72


# Updating stop words

In [39]:
stop_words = get_stop_words('Arabic_stop_word.txt')

In [40]:
dataset["article"]=dataset["article"].apply(lambda x: [item for item in x if item not in stop_words])

In [41]:
word_vectorizer_uni = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    stop_words=stop_words,
    ngram_range=(1, 1),
    max_features =1000)

In [42]:
unigramdataGet= word_vectorizer_uni.fit_transform(dataset['article'].astype('str'))
unigramdataGet = unigramdataGet.toarray()
vocab = word_vectorizer_uni.get_feature_names()
unigramdata=pd.DataFrame(np.round(unigramdataGet, 1), columns=vocab)
unigramdata[unigramdata>0] = 1

In [44]:
X=unigramdata
y=dataset['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=100)

In [45]:
svc=LinearSVC(C=1, max_iter=500)
svc= svc.fit(X_train , y_train)

y_pred1 = svc.predict(X_test)
dm=svc.score(X_test, y_test)
print('Accuracy score= {:.2f}'.format(svc.score(X_test, y_test)))

Accuracy score= 0.93


In [46]:
nab=GaussianNB(var_smoothing=1e-08)
nab= nab.fit(X_train , y_train)

y_pred1 = nab.predict(X_test)
nb=nab.score(X_test, y_test)
print('Accuracy score= {:.2f}'.format(nab.score(X_test, y_test)))

Accuracy score= 0.89


In [47]:
lor=LogisticRegression(C=2, max_iter=100)
lor= lor.fit(X_train , y_train)

y_pred1 = lor.predict(X_test)
lr=lor.score(X_test, y_test)
print('Accuracy score= {:.2f}'.format(lor.score(X_test, y_test)))

Accuracy score= 0.93


In [48]:
raf=RandomForestClassifier(min_samples_leaf=20, min_samples_split=20,random_state=100)
raf= raf.fit(X_train , y_train)

y_pred1 = raf.predict(X_test)
rf=raf.score(X_test, y_test)
print('Accuracy score= {:.2f}'.format(raf.score(X_test, y_test)))

Accuracy score= 0.86


In [49]:
Ens = VotingClassifier( estimators= [('SVM',svc),('NB',nab),('Log',lor),('raf',raf)], voting = 'hard')
Ens= Ens.fit(X_train , y_train)

y_pred1 = Ens.predict(X_test)
en=Ens.score(X_test, y_test)
print('Accuracy score= {:.2f}'.format(Ens.score(X_test, y_test)))

Accuracy score= 0.93


In [4]:
def train_predict (X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=100)
    
    svc=LinearSVC(C=1, max_iter=500)
    svc= svc.fit(X_train , y_train)
    y_pred1 = svc.predict(X_test)
    dm=svc.score(X_test, y_test)
    
    nab=GaussianNB(var_smoothing=1e-08)
    nab= nab.fit(X_train , y_train)
    y_pred2 = nab.predict(X_test)
    nb=nab.score(X_test, y_test)
    
    lor=LogisticRegression(C=2, max_iter=100)
    lor= lor.fit(X_train , y_train)
    y_pred3 = lor.predict(X_test)
    lr=lor.score(X_test, y_test)
    
    raf=RandomForestClassifier(min_samples_leaf=20, min_samples_split=20,random_state=100)
    raf= raf.fit(X_train , y_train)
    y_pred4 = raf.predict(X_test)
    rf=raf.score(X_test, y_test)
    
    Ens = VotingClassifier( estimators= [('SVM',svc),('NB',nab),('Log',lor),('raf',raf)], voting = 'hard')
    Ens= Ens.fit(X_train , y_train)
    y_pred5 = Ens.predict(X_test)
    en=Ens.score(X_test, y_test)
    
    
    print('Accuracy score for SVC= {:.2f}'.format(dm))  
    print('Accuracy score for NB= {:.2f}'.format(nb))
    print('Accuracy score for LR= {:.2f}'.format(lr))
    print('Accuracy score for RF= {:.2f}'.format(rf))
    print('Accuracy score for EN= {:.2f}'.format(en))

# Scraping more data 

In [51]:
dataset = pd.read_csv("data_4.csv")
dataset.drop(['Unnamed: 0'], axis=1)

,article,label
0,أكد عادل عقل، الخبير التحكيمى، أن الحكم الجوات...,sports
1,كشف محمد أسامة، رئيس الجهاز الطبي بالزمالك، حق...,sports
2,أكد طارق قنديل، عضو مجلس إدارة النادى الأهلي ،...,sports
3,احتفل مؤمن زكريا، لاعب الأهلي، بقوز المارد الأ...,sports
4,وصلت بعثة الفريق الأول لكرة القدم بالنادي الا...,sports
...,...,...
22885,قال الجنرال المتقاعد بالجيش الأمريكي لويد أوست...,world
22886,قال الرئيس الأمريكى المنتهية ولايته، دونالد تر...,world
22887,وصل منذ قليل الرئيس الأمريكي المنتخب جو بادين ...,world
22888,أكد رئيس نيجيريا محمد بخاري اليوم الثلاثاء أهم...,world


In [52]:
dataset['label'].unique()

array(['sports', 'economics', 'politics', 'celebrity', 'tech',
       'accidents', 'world_sports', 'health', 'arab', 'world'],
      dtype=object)

In [53]:
tokenizer = RegexpTokenizer(r'\w+')
dataset["article"] = dataset["article"].apply(tokenizer.tokenize)
dataset = dataset.drop(['Unnamed: 0'], axis=1)
dataset.head()

,article,label
0,"[أكد, عادل, عقل, الخبير, التحكيمى, أن, الحكم, ...",sports
1,"[كشف, محمد, أسامة, رئيس, الجهاز, الطبي, بالزما...",sports
2,"[أكد, طارق, قنديل, عضو, مجلس, إدارة, النادى, ا...",sports
3,"[احتفل, مؤمن, زكريا, لاعب, الأهلي, بقوز, المار...",sports
4,"[وصلت, بعثة, الفريق, الا, ول, لكرة, القدم, بال...",sports


In [54]:
dataset["article"]=dataset["article"].apply(lambda x: [item for item in x if item not in stop_words])

In [55]:
dataset['article'].head()

0    [عادل, عقل, الخبير, التحكيمى, الحكم, الجواتيما...
1    [أسامة, الجهاز, الطبي, بالزمالك, حقيقة, الأنبا...
2    [طارق, قنديل, عضو, مجلس, إدارة, النادى, الأهلي...
3    [احتفل, مؤمن, زكريا, لاعب, الأهلي, بقوز, المار...
4    [وصلت, بعثة, الفريق, ول, لكرة, القدم, بالنادي,...
Name: article, dtype: object

In [56]:
all_words_sports = [word for tokens in dataset[dataset['label']=='sports']['article'] for word in tokens]
counter = Counter(all_words_sports)
counter.most_common(25)

[('الزمالك', 549),
 ('لكرة', 478),
 ('العالم', 453),
 ('الفريق', 452),
 ('الأهلي', 422),
 ('اليد', 387),
 ('الأهلى', 331),
 ('الكرة', 331),
 ('فريق', 315),
 ('مباراة', 293),
 ('منتخب', 293),
 ('الأول', 267),
 ('المدير', 266),
 ('كأس', 256),
 ('لاعب', 253),
 ('بطولة', 249),
 ('الجولة', 233),
 ('المباراة', 229),
 ('الفني', 223),
 ('الدوري', 221),
 ('بيراميدز', 220),
 ('للأندية', 192),
 ('مباريات', 192),
 ('الدورى', 181),
 ('القدم', 180)]

In [57]:
all_words_economics = [word for tokens in dataset[dataset['label']=='economics']['article'] for word in tokens]
counter = Counter(all_words_economics)
counter.most_common(25)

[('تعاملات', 476),
 ('أسعار', 424),
 ('جلسة', 364),
 ('المصرية', 351),
 ('المصرى', 346),
 ('مستوى', 312),
 ('بنسبة', 310),
 ('سعر', 277),
 ('جلسات', 252),
 ('المؤشر', 235),
 ('المالية', 202),
 ('التجارية', 198),
 ('الذهب', 186),
 ('وزير', 185),
 ('الاثنين', 184),
 ('للبيع', 181),
 ('ليغلق', 180),
 ('للشراء', 179),
 ('لسوق', 179),
 ('الدكتور', 175),
 ('البنك', 167),
 ('تراجع', 161),
 ('الأحد', 160),
 ('الجنيه', 155),
 ('البنوك', 154)]

In [58]:
all_words_politics = [word for tokens in dataset[dataset['label']=='politics']['article'] for word in tokens]
counter = Counter(all_words_politics)
counter.most_common(25)

[('النواب', 1225),
 ('مجلس', 795),
 ('لجنة', 384),
 ('قانون', 377),
 ('بمجلس', 356),
 ('النائب', 350),
 ('المستشار', 339),
 ('الدكتور', 337),
 ('العامة', 309),
 ('وزير', 302),
 ('عضو', 298),
 ('برئاسة', 285),
 ('لسنة', 267),
 ('رقم', 265),
 ('المجلس', 238),
 ('الجلسة', 201),
 ('لمجلس', 200),
 ('القانون', 182),
 ('بشأن', 166),
 ('الأحزاب', 150),
 ('شباب', 145),
 ('اللجنة', 143),
 ('حزب', 142),
 ('الحكومة', 141),
 ('جبالى', 140)]

In [59]:
all_words_celebrity = [word for tokens in dataset[dataset['label']=='celebrity']['article'] for word in tokens]
counter = Counter(all_words_celebrity)
counter.most_common(25)

[('الفنان', 471),
 ('الفنانة', 435),
 ('النجمة', 401),
 ('النجم', 387),
 ('مسلسل', 260),
 ('فيلم', 195),
 ('أحمد', 177),
 ('موقع', 174),
 ('تصوير', 167),
 ('الجديد', 162),
 ('صورة', 148),
 ('جديدة', 135),
 ('أغنية', 126),
 ('حسابها', 123),
 ('العالمية', 120),
 ('المطرب', 111),
 ('الراحل', 107),
 ('الكبير', 105),
 ('إنستجرام', 103),
 ('عمر', 102),
 ('رمضان', 102),
 ('حسابه', 102),
 ('انستجرام', 100),
 ('بعيد', 98),
 ('بموقع', 86)]

In [60]:
all_words_tech = [word for tokens in dataset[dataset['label']=='tech']['article'] for word in tokens]
counter = Counter(all_words_tech)
counter.most_common(25)

[('شركة', 571),
 ('أبل', 316),
 ('جوجل', 211),
 ('جديدة', 170),
 ('الفضاء', 156),
 ('تطبيق', 152),
 ('العالم', 139),
 ('الجديدة', 131),
 ('بوك', 124),
 ('الشركة', 117),
 ('لشركة', 114),
 ('أكثر', 113),
 ('جديد', 111),
 ('الأمريكية', 109),
 ('iPhone', 108),
 ('فيس', 105),
 ('الرئيس', 102),
 ('المتحدة', 99),
 ('الخاصة', 96),
 ('المستخدمين', 93),
 ('تقرير', 92),
 ('مجموعة', 92),
 ('الهواتف', 91),
 ('موقع', 90),
 ('الذكية', 90)]

In [61]:
all_words_accidents = [word for tokens in dataset[dataset['label']=='accidents']['article'] for word in tokens]
counter = Counter(all_words_accidents)
counter.most_common(25)

[('محكمة', 332),
 ('العامة', 327),
 ('القاهرة', 308),
 ('الأمن', 251),
 ('أمن', 242),
 ('النيابة', 242),
 ('الجيزة', 234),
 ('ضبط', 220),
 ('قررت', 199),
 ('أجهزة', 194),
 ('الأمنية', 190),
 ('الداخلية', 185),
 ('رجال', 184),
 ('ذمة', 181),
 ('نيابة', 180),
 ('اللواء', 172),
 ('جنايات', 171),
 ('داخل', 163),
 ('برئاسة', 158),
 ('بمديرية', 157),
 ('حبس', 155),
 ('المستشار', 153),
 ('بمحافظة', 152),
 ('شرطة', 145),
 ('التحقيقات', 139)]

In [62]:
all_words_world_sports = [word for tokens in dataset[dataset['label']=='world_sports']['article'] for word in tokens]
counter = Counter(all_words_world_sports)
counter.most_common(25)

[('فريق', 780),
 ('ملعب', 523),
 ('الدوري', 409),
 ('المباراة', 382),
 ('كأس', 369),
 ('الإنجليزي', 314),
 ('منافسات', 299),
 ('ليفربول', 295),
 ('نظيره', 269),
 ('النجم', 267),
 ('الجولة', 266),
 ('يونايتد', 257),
 ('مانشستر', 253),
 ('المدير', 243),
 ('مواجهة', 230),
 ('العالم', 224),
 ('لفريق', 216),
 ('مضيفه', 212),
 ('برشلونة', 209),
 ('الفريقين', 206),
 ('مدريد', 202),
 ('لاعب', 201),
 ('القدم', 193),
 ('نادي', 182),
 ('الإسباني', 177)]

In [63]:
all_words_health = [word for tokens in dataset[dataset['label']=='health']['article'] for word in tokens]
counter = Counter(all_words_health)
counter.most_common(25)

[('فيروس', 625),
 ('لقاح', 345),
 ('دراسة', 325),
 ('الأمريكية', 239),
 ('جديدة', 194),
 ('الإصابة', 179),
 ('أكثر', 177),
 ('المتحدة', 166),
 ('الصحة', 148),
 ('لفيروس', 145),
 ('الأمراض', 142),
 ('بفيروس', 138),
 ('العالم', 136),
 ('الصحية', 134),
 ('شركة', 133),
 ('تقرير', 131),
 ('البريطانية', 125),
 ('جامعة', 125),
 ('الأشخاص', 107),
 ('العديد', 100),
 ('موقع', 99),
 ('الأطفال', 97),
 ('الجسم', 95),
 ('الجديدة', 91),
 ('فايزر', 90)]

In [64]:
all_words_arab = [word for tokens in dataset[dataset['label']=='arab']['article'] for word in tokens]
counter = Counter(all_words_arab)
counter.most_common(25)

[('وزارة', 399),
 ('العربية', 372),
 ('الصحة', 349),
 ('بفيروس', 282),
 ('إصابة', 275),
 ('وزير', 262),
 ('جديدة', 254),
 ('عاجل', 230),
 ('قناة', 225),
 ('مجلس', 221),
 ('قليل', 217),
 ('تسجيل', 217),
 ('السعودية', 216),
 ('المستجد', 205),
 ('حالة', 199),
 ('وفقا', 173),
 ('الخارجية', 172),
 ('الوزراء', 170),
 ('المتحدة', 160),
 ('فيروس', 142),
 ('لخبر', 134),
 ('الماضية', 134),
 ('بثته', 133),
 ('الدكتور', 131),
 ('الحكومة', 122)]

In [65]:
all_words_world = [word for tokens in dataset[dataset['label']=='world']['article'] for word in tokens]
counter = Counter(all_words_world)
counter.most_common(25)

[('الرئيس', 572),
 ('بايدن', 369),
 ('المتحدة', 357),
 ('الأمريكي', 332),
 ('جو', 332),
 ('الأمريكية', 319),
 ('فيروس', 293),
 ('بفيروس', 252),
 ('جديدة', 251),
 ('المستجد', 219),
 ('إصابة', 211),
 ('وفقا', 201),
 ('وزارة', 201),
 ('الولايات', 192),
 ('الصحة', 191),
 ('الأمريكى', 184),
 ('وزير', 178),
 ('الخارجية', 175),
 ('قناة', 173),
 ('عاجل', 170),
 ('كوفيد', 164),
 ('البلاد', 163),
 ('العربية', 162),
 ('قليل', 157),
 ('ترامب', 155)]

In [66]:
word_vectorizer_uni = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    stop_words=stop_words,
    ngram_range=(1, 1),
    max_features =1000)

In [67]:
unigramdataGet= word_vectorizer_uni.fit_transform(dataset['article'].astype('str'))
unigramdataGet = unigramdataGet.toarray()
vocab = word_vectorizer_uni.get_feature_names()
unigramdata=pd.DataFrame(np.round(unigramdataGet, 1), columns=vocab)
unigramdata[unigramdata>0] = 1

In [69]:
X=unigramdata
y=dataset['label']

train_predict(X,y)

Accuracy score= 0.83
Accuracy score= 0.71
Accuracy score= 0.84
Accuracy score= 0.71
Accuracy score= 0.83


# Updating stop words 

In [70]:
stop_words = get_stop_words('Arabic_stop_word.txt')
dataset["article"]=dataset["article"].apply(lambda x: [item for item in x if item not in stop_words])

In [71]:
word_vectorizer_uni = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    stop_words=stop_words,
    ngram_range=(1, 1),
    max_features =1000)

In [72]:
unigramdataGet= word_vectorizer_uni.fit_transform(dataset['article'].astype('str'))
unigramdataGet = unigramdataGet.toarray()
vocab = word_vectorizer_uni.get_feature_names()
unigramdata=pd.DataFrame(np.round(unigramdataGet, 1), columns=vocab)
unigramdata[unigramdata>0] = 1

In [76]:
X=unigramdata
y=dataset['label']

train_predict(X,y)

Accuracy score for SVC= 0.81
Accuracy score for NB= 0.70
Accuracy score for LR= 0.82
Accuracy score for RF= 0.72
Accuracy score for EN= 0.81


# Using Stemming 

In [8]:
dataset = pd.read_csv("data_4.csv")
dataset.drop(['Unnamed: 0'], axis=1)
tokenizer = RegexpTokenizer(r'\w+')
dataset["article"] = dataset["article"].apply(tokenizer.tokenize)
dataset = dataset.drop(['Unnamed: 0'], axis=1)
dataset["article"]=dataset["article"].apply(lambda x: [item for item in x if item not in stop_words])
dataset.head()

,article,label
0,"[عادل, عقل, الخبير, التحكيمى, الحكم, الجواتيما...",sports
1,"[أسامة, الجهاز, الطبي, بالزمالك, حقيقة, الأنبا...",sports
2,"[طارق, قنديل, عضو, مجلس, إدارة, النادى, الأهلي...",sports
3,"[احتفل, مؤمن, زكريا, الأهلي, بقوز, المارد, الأ...",sports
4,"[وصلت, بعثة, الفريق, ول, لكرة, بالنادي, هلي, ل...",sports


In [9]:
import nltk
from nltk.stem.isri import ISRIStemmer

st = ISRIStemmer()

dataset['article'] = dataset['article'].apply(lambda x : [st.stem(item) for item in x])
dataset.head()

,article,label
0,"[عدل, عقل, خبر, حكيمى, حكم, جواتيمالى, ظهر, ست...",sports
1,"[سمة, جهز, طبي, زمل, حقق, باء, دول, فتر, صبة, ...",sports
2,"[طرق, قنديل, عضو, جلس, درة, ندى, اهل, نهم, يوج...",sports
3,"[حفل, ؤمن, زكر, اهل, بقز, ارد, حمر, دحل, قطر, ...",sports
4,"[وصل, بعث, فرق, ول, لكر, ندي, هلي, لى, مقر, قم...",sports


In [18]:
def train_predict_pipeline(X, y):
    nb_clf = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', MultinomialNB())])
    svc_clf = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', LinearSVC())])
    lr_clf = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', LogisticRegression())])
    rf_clf = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', RandomForestClassifier())])
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=100)
    
    nb_clf = nb_clf.fit(X_train, y_train)
    predicted_nb = nb_clf.predict(X_test)
    
    svc_clf = svc_clf.fit(X_train, y_train)
    predicted_svc = svc_clf.predict(X_test)
    
    lr_clf = lr_clf.fit(X_train, y_train)
    predicted_lr = lr_clf.predict(X_test)
    
    rf_clf = rf_clf.fit(X_train, y_train)
    predicted_rf = rf_clf.predict(X_test)
    
    print('Accuracy score for MNB= {:.2f}'.format(np.mean(predicted_nb == y_test)))
    print('Accuracy score for SVC= {:.2f}'.format(np.mean(predicted_svc == y_test)))  
    print('Accuracy score for LR= {:.2f}'.format(np.mean(predicted_lr == y_test)))
    print('Accuracy score for RF= {:.2f}'.format(np.mean(predicted_rf == y_test)))
    

In [17]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

train_predict_pipeline(dataset['article'].astype('str'), dataset['label'])

Accuracy score for MNB= 0.88
Accuracy score for SVC= 0.90
Accuracy score for LR= 0.89
Accuracy score for RF= 0.85


# Using Grid Search to get the best choise of parameters 

In [10]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV

X_train, X_test, y_train, y_test = train_test_split(dataset['article'].astype('str'), dataset['label'], test_size=0.30, random_state=100)

In [11]:
pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', LinearSVC()),
])


parameters = {
    'vect__max_df': (0.5, 1.0),
    'vect__max_features': (5000, 10000),
    'vect__ngram_range': ((1, 1), (1, 2)),  
    'tfidf__use_idf': (True, False),
    'tfidf__norm': ('l1', 'l2'),
    'clf__max_iter': (20, 100, 1000),
    'clf__penalty': ('l2', 'l1'),
    'clf__loss': ('hinge', 'squared_hinge')
}


grid_search = GridSearchCV(pipeline, parameters, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

print("best params: " + str(grid_search.best_params_))
print("best scores: " + str(grid_search.best_score_))

best params: {'clf__loss': 'hinge', 'clf__max_iter': 100, 'clf__penalty': 'l2', 'tfidf__norm': 'l2', 'tfidf__use_idf': True, 'vect__max_df': 0.5, 'vect__max_features': 10000, 'vect__ngram_range': (1, 2)}
best scores: 0.9043873784913943


In [18]:
from sklearn.metrics import accuracy_score

acc = accuracy_score(y_test, grid_search.predict(X_test))
print("Accuracy: {:.4%}".format(acc))

Accuracy: 90.3306%
